In [ ]:
from utils import compare
from utils import visualization
import anndata
from gsea_api.molecular_signatures_db import GeneSets
import matplotlib.pyplot as plt
import numpy as np
import scipy
import multiprocessing

#### Import data

In [ ]:
%cd data/
immune_cell = anndata.read_h5ad('all_cells.h5ad')

In [ ]:
from gsea_api.molecular_signatures_db import GeneSets
immune_gene_set = GeneSets.from_gmt('c7.immunesigdb.v7.5.1.symbols.gmt')
immune_gs_name = [item.name for item in immune_gene_set.gene_sets]
immune_gene_set = [list(immune_gene_set.gene_sets[i].genes) for i in range(len(immune_gene_set.gene_sets))]
print(f"ImmuneSigDB database has {len(immune_gene_set)} gene sets.")
curated_cell_label = "Classical monocytes"

In [ ]:
subset_scRNAseq = immune_cell[(immune_cell.obs['Organ'] == "BLD")*(immune_cell.obs['Manually_curated_celltype'] == curated_cell_label)]

In [ ]:
subset_scRNAseq

In [ ]:
from imp import reload
reload(compare)

#### Running time comparisons among different models, and the associations among their similarity matrices.

In [ ]:
sample_size = [800, 1000, 1200, 1400]
gene_set_size = [50, 100, 200, 400, 600, 800, 1000]
clustering_metric = 'AMI'
num_core = multiprocessing.cpu_count()

np.random.seed(123)
for g_size in gene_set_size:
    ### Fix sampled gene sets 
    sampling = np.random.choice(len(immune_gene_set), size = g_size, replace = False)
    sampled_gs = [immune_gene_set[item] for item in sampling]
    sampled_gs_names = [immune_gs_name[item] for item in sampling]  
    for s_size in sample_size:
        
        sampling = np.random.choice(subset_scRNAseq.X.shape[0], size = s_size, replace = False)
        sampled_data = subset_scRNAseq[sampling,]

        exec(f'res_{s_size}_{g_size} = compare.run_duration_comparison_full(sampled_gs = sampled_gs, clustering_metric = clustering_metric, sampled_data = sampled_data, num_core = num_core)')
        exec(f'np.save("res_{s_size}_{g_size}.npy", res_{s_size}_{g_size})')
        if eval(f'np.load("res_{s_size}_{g_size}.npy", allow_pickle = True)') is not None:
            print(f'An experiment with sample size {s_size}, gene set size {g_size} finishes.')
            exec(f'del res_{s_size}_{g_size}')

$~$

$~$

#### Impacts of clustering metric and hyperparameter k on $\mathcal{S}_{\phi|\text{k-means}}$

In [ ]:
K_num = list(range(2,10,1))
_ = [K_num.append(item) for item in range(10,90,5)]

In [ ]:
hyperparameters = [{'n_clusters':k, 'random_state':0, 'n_init':10, 'max_iter':1000, 'tol':1e-4, 'algorithm':"lloyd"} for k in K_num]
### fix samples cells and gene sets
sample_size = [1200]
gene_set_size = [100]
np.random.seed(123)
sampling = np.random.choice(subset_scRNAseq.X.shape[0], size = sample_size[0], replace = False)
sampled_data = subset_scRNAseq[sampling,]
sampling = np.random.choice(len(immune_gene_set), size = gene_set_size[0], replace = False)
sampled_gs = [immune_gene_set[item] for item in sampling]
sampled_gs_names = [immune_gs_name[item] for item in sampling] 

In [ ]:
for clustering_metric in ["AMI", "NMI", "rand_adj"]:
    exec(f'hyper_kmeans = compare.run_duration_comparison_hyperparameter(sampled_gs = sampled_gs, sampled_data = sampled_data, worker = compare.hyperparameter_worker_kmeans, clustering_metric = clustering_metric, hyper_lst = hyperparameters, num_core = 60)')
    np.save(f"hyper_kmeans_{clustering_metric}.npy", hyper_kmeans)
    if eval(f'np.load("hyper_kmeans_{clustering_metric}.npy", allow_pickle = True)') is not None:
        print('Complete!')

In [ ]:
length = len(K_num)
metrics = ["AMI", "NMI", "rand_adj"]
for ns in metrics:
    exec(f'res_hyper_{ns} = np.load("hyper_kmeans_{ns}.npy", allow_pickle = True)')
    exec(f'cor_mat_{ns} = np.array([(lambda i,j: scipy.stats.pearsonr(res_hyper_{ns}[0][i][np.triu_indices_from(res_hyper_{ns}[0][i], k = 1)],\
                                            res_hyper_{ns}[0][j][np.triu_indices_from(res_hyper_{ns}[0][j], k = 1)])[0])(i,j) for i in range(length)\
                  for j in range(length)]).reshape(length, length)')
    exec(f'cor_RV_{ns} = np.array([(lambda i: scipy.stats.pearsonr(res_hyper_{ns}[0][i][np.triu_indices_from(res_hyper_{ns}[0][i], k = 1)],\
                                           res_hyper_{ns}[2][np.triu_indices_from(res_hyper_{ns}[2], k = 1)])[0])(i) for i in range(length)])')
    exec(f'cor_mantel_{ns} = np.array([(lambda i: scipy.stats.pearsonr(res_hyper_{ns}[0][i][np.triu_indices_from(res_hyper_{ns}[0][i], k = 1)],\
                                           res_hyper_{ns}[3][np.triu_indices_from(res_hyper_{ns}[3], k = 1)])[0])(i) for i in range(length)])')

In [ ]:
### Gene set size = 100
cor_RV = np.array([eval(f'cor_RV_{ns}') for ns in metrics])
labels = ["AMI", "NMI", "ARI"]
fig = visualization.vis_hyper_kmeans(cor_RV, K_num, labels = labels , ref_name = "Compare with the modified RV coefficient", figsize=(12,6), anno_font_size = 18, title_size = 18)
fig.savefig("cor_adjusted_RV.png", dpi = 300)

In [ ]:
### Gene set size = 100
cor_mantel = np.array([eval(f'cor_mantel_{ns}') for ns in metrics])
labels = ["AMI", "NMI", "ARI"]
fig = visualization.vis_hyper_kmeans(cor_mantel, K_num, labels = labels , ref_name = "Compare with the Mantel coefficient", figsize=(12,6), anno_font_size = 18, title_size = 18)
fig.savefig("cor_Mantel.png", dpi = 300)

In [ ]:
### Gene set size = 100
fig_hyper = visualization.vis_heatmap(cor_mat_AMI, labels = K_num, figsize=(20,20), title = "With different hyperparameters (K)",  legend_title = "Pearson correlation coefficient",\
                                      x_rotation = 0, y_rotation = 0, ticks = np.arange(0.5,length+0.5,1), \
                                      title_size = 20, anno_font_size = 18 )
fig_hyper.savefig("kmeans_hyperparameter.png", dpi = 300)

In [ ]:
### Gene set size = 100
fig_hyper = visualization.vis_heatmap(cor_mat_rand_adj, labels = K_num, figsize=(20,20), title = "With different hyperparameters (K)",  legend_title = "Pearson correlation coefficient",\
                                      x_rotation = 0, y_rotation = 0, ticks = np.arange(0.5,length+0.5,1), \
                                      title_size = 20, anno_font_size = 18 )

In [ ]:
fig_hyper.savefig("hyper_concor/kmeans_hyperparameter_rand_adj.png", dpi = 300)

In [ ]:
### Gene set size = 100
fig_hyper = visualization.vis_heatmap(cor_mat_NMI, labels = K_num, figsize=(20,20), title = "With different hyperparameters (K)",  legend_title = "Pearson correlation coefficient",\
                                      x_rotation = 0, y_rotation = 0, ticks = np.arange(0.5,length+0.5,1), \
                                      title_size = 20, anno_font_size = 18 )

In [ ]:
fig_hyper.savefig("hyper_concor/kmeans_hyperparameter_NMI.png", dpi = 300)

paired Wilcoxon signed-rank test 

In [ ]:
### number of gene sets : 100
### AMI vs ARI
scipy.stats.wilcoxon(x = cor_RV[0,:], y= cor_RV[2,:], zero_method='wilcox', correction=False, alternative='two-sided')

In [ ]:
### AMI vs NMI
scipy.stats.wilcoxon(x = cor_RV[0,:], y= cor_RV[1,:], zero_method='wilcox', correction=False, alternative='two-sided')

In [ ]:
### ARI vs NMI
scipy.stats.wilcoxon(x = cor_RV[2,:], y= cor_RV[1,:], zero_method='wilcox', correction=False, alternative='two-sided')

In [ ]:
### number of gene sets : 100
### AMI vs ARI
scipy.stats.wilcoxon(x = cor_mantel[0,:], y= cor_mantel[2,:], zero_method='wilcox', correction=False, alternative='two-sided')

In [ ]:
### AMI vs NMI
scipy.stats.wilcoxon(x = cor_mantel[0,:], y= cor_mantel[1,:], zero_method='wilcox', correction=False, alternative='two-sided')

In [ ]:
### ARI vs NMI
scipy.stats.wilcoxon(x = cor_mantel[2,:], y= cor_mantel[1,:], zero_method='wilcox', correction=False, alternative='two-sided')

$~$

$~$

In [ ]:
%cd res_comparison

In [ ]:
sample_size = [800, 1000, 1200, 1400]
gene_set_size = [50, 100, 200, 400, 600, 800, 1000]

In [ ]:
fig = visualization.vis_surface(sample_size = sample_size, gs_size = gene_set_size, unit = 'hour',\
                          figsize = (12, 10), dpi=300,alpha = 0.75, \
                          legend_pos = (1.0, 1.0), elevation_rotation = [20, -60])

In [ ]:
fig.savefig('program_running_time_comparison.png', dpi = 300)

In [ ]:
fig_RV, _ = visualization.vis_surface_correlation(sample_size = sample_size, gs_size = gene_set_size,  \
                          figsize = (12, 10), dpi=300,alpha = 0.75, \
                          legend_pos = (1.0, 1.0), elevation_rotation = [20, -60])

In [ ]:
fig_RV.savefig('Pearson_RV.png', dpi = 300)

In [ ]:
fig_mantel, res_mantel = visualization.vis_surface_correlation(sample_size = sample_size, gs_size = gene_set_size, reference = "Mantel",  \
                          figsize = (12, 10), dpi=300,alpha = 0.75, \
                          legend_pos = (1.0, 1.0), elevation_rotation = [20, -60])

In [ ]:
fig_mantel.savefig('pearson_mantel.png', dpi = 300)

$~$

In [ ]:
### Example
x = 3
y = 6
print(f'Sample size is {sample_size[x]}, gene set size is {gene_set_size[y]}')
methods = ["RV_mod", "Leiden", "BGM", "Kmeans", "Jaccard_mod", "Mantel"]

In [ ]:
fig = visualization.vis_heatmap(res_mantel[x*len(gene_set_size) + y], labels = methods, figsize=(16,16), title = "Among similarity matrices",  legend_title = "Pearson correlation coefficient",\
                                      x_rotation = 0, y_rotation = 0, ticks = np.arange(0.5,6+0.5,1), \
                                      title_size = 20, anno_font_size = 18 )

In [ ]:
fig.savefig("cor_sim_mat.png", dpi = 300)

$~$

In [ ]:
### Example
res_1400_1000 = np.load('res_1400_1000.npy', allow_pickle = True).tolist()

In [ ]:
sim_RV_mod = res_1400_1000[1][0]
sim_kmeans = res_1400_1000[1][3]
sim_mantel = res_1400_1000[1][5]

In [ ]:
fig = visualization.vis_scatter(sim_RV_mod[np.triu_indices_from(sim_RV_mod, k = 1)],\
                          sim_kmeans[np.triu_indices_from(sim_kmeans, k = 1)], \
                          "modified RV", f'S$_{{Φ|k-means, AMI}}$', para_jointplot = {'kind':'hex', 'space':0.7, 'marginal_kws':dict(bins=30)},\
               anno_font_size = 16, height = 8, ratio = 10)

In [ ]:
fig.savefig("RV_mod_kmeans_1400_1000_cor.png", dpi = 300)

In [ ]:
scipy.stats.pearsonr(sim_RV_mod[np.triu_indices_from(sim_RV_mod, k = 1)], \
                     sim_kmeans[np.triu_indices_from(sim_kmeans, k = 1)])

In [ ]:
fig = visualization.vis_scatter(sim_mantel[np.triu_indices_from(sim_mantel, k = 1)],\
                          sim_kmeans[np.triu_indices_from(sim_kmeans, k = 1)], \
                          "Mantel", f'S$_{{Φ|k-means, AMI}}$', para_jointplot = {'kind':'hex', 'space':0.7, 'marginal_kws':dict(bins=30)},\
               anno_font_size = 16, height = 8, ratio = 10)

In [ ]:
fig.savefig("Mantel_kmeans_1400_1000_cor.png", dpi = 300)

In [ ]:
scipy.stats.pearsonr(sim_mantel[np.triu_indices_from(sim_mantel, k = 1)], \
                     sim_kmeans[np.triu_indices_from(sim_kmeans, k = 1)])

$~$

#### Average ratios

In [ ]:
time_lst = []
for sample in sample_size:
    time_lst.append(np.load(f'res_{sample}_1000.npy', allow_pickle = True).tolist()[0])   
RV_mod_time = np.array([item[0][1] for item in time_lst])
mantel_time = np.array([item[5][1] for item in time_lst])
kmeans_time = np.array([item[3][1] for item in time_lst])
leiden_time = np.array([item[1][1] for item in time_lst])
BGM_time = np.array([item[2][1] for item in time_lst])

In [ ]:
np.mean(RV_mod_time/kmeans_time)

In [ ]:
np.mean(RV_mod_time/BGM_time)

In [ ]:
np.mean(mantel_time/kmeans_time)

In [ ]:
np.mean(mantel_time/BGM_time)